In [27]:
import requests
import pandas as pd
import csv
import json
from pprint import pprint as print


from uk_election_ids.election_ids import IdBuilder

In [2]:
# ELECTION TYPES
url_et = 'http://elections.democracyclub.org.uk/api/election_types/'
r_et = requests.get(url_et)
data_et = r_et.json()

electionTypes = {}

for electionTypeDict in data_et['results']:
    electionTypes[electionTypeDict['election_type']] = electionTypeDict['name']

In [3]:
# ELECTION SUBTYPES
url_est = 'http://elections.democracyclub.org.uk/api/election_subtypes/'
r_est = requests.get(url_est)
data_est = r_est.json()
df_est = pd.DataFrame(data_est['results'])

In [4]:
# ORGANISATIONS
url_o = 'https://elections.democracyclub.org.uk/api/organisations'
organisations = {}

while url_o:
    r_o = requests.get(url_o)
    data_o = r_o.json()
    for orgDict in data_o['results']:
        organisations[orgDict['slug']] = orgDict
    url_o = data_o['next']

In [8]:
# Get all future elections data --> df_ee

url_ee = 'https://elections.democracyclub.org.uk/api/elections.json?future=1'
df_ee = pd.DataFrame()

while url_ee:
    print(url_ee)
    res_ee = requests.get(url_ee)
    res_ee.raise_for_status()
    data_ee = res_ee.json()
    df_ee_subset = pd.DataFrame(data_ee['results'])
    df_ee = pd.concat([df_ee, df_ee_subset], ignore_index=True)
    url_ee = data_ee['next']
print('Pulled ' + str(len(df_ee)) + ' elections from the Every Election API')

'https://elections.democracyclub.org.uk/api/elections.json?future=1'
'Pulled 60 elections from the Every Election API'


In [11]:
# print(df_ee.iloc[55])
# print(df_ee.iloc[55]['division'])
# print(df_ee.iloc[55]['election_type'])
# print(df_ee.iloc[55]['organisation'])

In [12]:
# BUILD UP A DATASET OF BALLOTS

ballots = {}

for i, row in df_ee.iterrows():

    # Heiarchy of election data: 
    #    Election type > [Election subtype] > [Organisation] > [Division] > [By-election] > Date polls open

    # We want to create a nested dictionary that follows this hierarchy. I.e. a dictionary of election types,
    # each containing a dictionary of organisations, each containing a dictionary of ballots (division.date_poll_opens).
    
    # Later on we will add a dictionary of candidates for each ballot. 
    
    # In the election object we pulled from elections.dc, if group_type is none then we are at the bottom of 
    # the hiearchy, so we know this is a ballot (and the election_id is therefore our ballot_id)
    
    if row['group_type'] is None:
        
        # Election Type
        electionType = row['election_type']['election_type']
        if electionType not in ballots:
            ballots[electionType] = {}
        
        # Organisation
        organisationId = row['group']
        if organisationId not in ballots[electionType]:
            ballots[electionType][organisationId] = {
                'organisation_slug': row['organisation']['slug'],  # We need this to lookup to our master list of orgs
                'ballots': {}
            }
        
        # Ballots
        ballotId = row['election_id']
        ballots[electionType][organisationId]['ballots'][ballotId] = {
            'ballot_id': ballotId,
            'ynr_election_id': organisationId,
            'election_title': row['election_title'],
            'election_type': electionType,
            'election_type_name': row['election_type']['name'],
            'elected_role': row['elected_role'],
            'division_name': row['division']['name'],
            'division_id': row['division']['official_identifier'],
            'seats_total': row['division']['seats_total'],
            'organisation_name': row['organisation']['common_name'],
            'organisation_type': row['organisation']['organisation_type'],            
            'organisation_subtype': row['organisation']['organisation_subtype'],
            'poll_open_date': row['poll_open_date']
        }

In [19]:
# GET CANDIDATE DATA FOR THE ORGANISATIONS OF ALL BALLOTS

# We pull candidate data from YourNextRepresentative (YNR, candidates.democracyclub.org.uk), 
# which identifies elections with the organisation ID (not the ballot ID) and a post ID, so we need to 
# call the candidates API using the organisation_id.
# Of course, we will have multiple ballots for each organisation. So to avoid hitting the API every time, 
# we'll build up a standalone list of candidates and thus only access the API for new orgIds

candidates = {}

for electionType in ballots:
    for orgId in ballots[electionType]:
        for ballotId in ballots[electionType][orgId]['ballots']:
            
            ballot = ballots[electionType][orgId]['ballots'][ballotId]
    
            if orgId not in candidates:

                candidates[orgId] = {}

                url_ynr = "https://candidates.democracyclub.org.uk/media/candidates-%s.csv" % (orgId)
                print(url_ynr)
                
                try:
                    res_ynr = requests.get(url_ynr)
                    res_ynr.raise_for_status()
                    decoded_data_ynr = res_ynr.content.decode('utf-8')
                    data_ynr = list(csv.DictReader(decoded_data_ynr.splitlines(), delimiter=','))
                    print('Pulled ' + str(len(data_ynr)) + ' candidates for ' + orgId)
                    for candidate_ynr in data_ynr:
                        candidates[orgId][candidate_ynr['id']] = dict(candidate_ynr)
                except requests.exceptions.HTTPError:
                    print('Pulled 0 candidates for ' + orgId)

'https://candidates.democracyclub.org.uk/media/candidates-local.bury.2018-08-16.csv'
'Pulled 0 candidates for local.bury.2018-08-16'
'https://candidates.democracyclub.org.uk/media/candidates-local.city-of-london.2018-08-30.csv'
'Pulled 0 candidates for local.city-of-london.2018-08-30'
'https://candidates.democracyclub.org.uk/media/candidates-local.cornwall.2018-08-23.csv'
'Pulled 0 candidates for local.cornwall.2018-08-23'
'https://candidates.democracyclub.org.uk/media/candidates-local.cornwall.2018-08-09.csv'
'Pulled 0 candidates for local.cornwall.2018-08-09'
'https://candidates.democracyclub.org.uk/media/candidates-local.east-devon.2018-09-20.csv'
'Pulled 0 candidates for local.east-devon.2018-09-20'
'https://candidates.democracyclub.org.uk/media/candidates-local.east-hertfordshire.2018-08-23.csv'
'Pulled 0 candidates for local.east-hertfordshire.2018-08-23'
'https://candidates.democracyclub.org.uk/media/candidates-local.fife.2018-09-06.csv'
'Pulled 0 candidates for local.fife.2018-

In [20]:
# LINK CANDIDATE DATA TO BALLOTS

matchedCandidates = []

for electionType in ballots:
    for orgId in ballots[electionType]:
        for ballotId in ballots[electionType][orgId]['ballots']:  
            ballot = ballots[electionType][orgId]['ballots'][ballotId]
            ballot['candidates'] = {}
            orgCandidates = candidates[orgId]
            for candidateId in orgCandidates:
                if ballot['division_name'] == orgCandidates[candidateId]['post_label']:
                    ballot['candidates'][candidateId] = orgCandidates[candidateId]
                    matchedCandidates.append(candidateId)

In [28]:
# OUTPUT TO JSON FILE

with open('upcoming_elections.json', 'w') as outfile:
    json.dump(ballots, outfile)

In [25]:
# PRINT OUT A LIST OF ELECTIONS

for electionType in ballots:
    
    print('')
    print(electionTypes[electionType])
    
    for orgId in ballots[electionType]:

        # Our master list of organistions is keyed with the org slug, which we have saved in our ballot record
        # So use this to lookup the election name from the organisation master data
        print('')
        print(organisations[ballots[electionType][orgId]['organisation_slug']]['election_name'])

        for ballotId in ballots[electionType][orgId]['ballots']:   
            
            ballot = ballots[electionType][orgId]['ballots'][ballotId]
            
            seatsCount = str(ballot['seats_total'])
            plural = ''
            if  ballot['seats_total'] is None:
                seatsCount = '[unknown number of]'
                plural = 's'
            elif int(ballot['seats_total']) > 1:
                plural = 's'

            print('  - ' + 
                  ballot['election_type_name'] + ' for ' + 
                  seatsCount + ' ' + 
                  ballot['elected_role'] + plural + ' in ' + 
                  ballot['organisation_name'] + "'s " + 
                  ballot['division_name'] + ' division, on ' +  
                  str(ballot['poll_open_date']))
            
            print('')
            if len(ballot['candidates']) == 0:
                print('No candidates in data yet')
                
            for candidateId in ballot['candidates']:
                
                can = ballot['candidates'][candidateId]
                gender = 'UNKNOWN'
                
                if can['gender'] is not None and can['gender'] != '':
                    gender = can['gender']
                
                print('    - ' + can['name'] + ' | ' + 
                      ' Gender: ' + gender + ' | ' + 
                      '@' + can['twitter_username'])

''
'Local elections'
''
'Bury local election'
("  - Local elections for [unknown number of] Local Councillors in Bury's East "
 'division, on 2018-08-16')
''
'No candidates in data yet'
''
'City of London local election'
('  - Local elections for [unknown number of] Local Councillors in City of '
 "London's Bread Street division, on 2018-08-30")
''
'No candidates in data yet'
''
'Cornwall local election'
("  - Local elections for [unknown number of] Local Councillors in Cornwall's "
 'Bude division, on 2018-08-23')
''
'No candidates in data yet'
''
'Cornwall local election'
("  - Local elections for [unknown number of] Local Councillors in Cornwall's "
 'Newquay Treviglas division, on 2018-08-09')
''
'No candidates in data yet'
''
'East Devon local election'
('  - Local elections for [unknown number of] Local Councillors in East '
 "Devon's Ottery St Mary Rural division, on 2018-09-20")
''
'No candidates in data yet'
''
'East Hertfordshire local election'
('  - Local elections for [unk